In [1]:
cd ../src/train_VAE/

/home/rfit/Telescope_Array/phd_work/src/train_VAE


In [2]:
import h5py as h5
import numpy as np
import os
import matplotlib.pyplot as plt
data_path = '/home3/rfit/Telescope_Array/phd_work/data/normed/pr_fe_q4_e1_0110_excl_sat_F_excl_geo_F.h5'

In [3]:
'''
mc_params (num_evs,10):
0. mc_event_num
1. mc_parttype (CORSIKA, 1 - gamma, 14 - proton, 5626 - Fe)
2. mc_corecounter, closest to core detector number
3. mc_E (for primaries other than photon energy is rescaled by 1/1.27, i.e. to proton FD energy scale)
4. mc_theta
5. mc_phi
6. mc_height_1st_inter, km
7. mc_xcore
8. mc_ycore
9. mc_border_distance, km 
'''
with h5.File(data_path,'r') as f:
    print('keys', list(f.keys()))
    test = f['test']
    keys = list(test.keys())
    mc_params = test['mc_params'][:]
    for k in keys:
        print(k, test[k].shape)

keys ['norm_param', 'test', 'train', 'val']
dt_mask (1045262, 2)
dt_params (26962300, 6)
ev_ids (1045262, 3)
ev_starts (1045263,)
mc_params (1045262, 10)
recos (1045262, 6)
wfs_flat (26962300, 128, 2)


In [14]:
import importlib
importlib.reload(pipline)

1.10.1+cu111
Using device: cuda


<module 'pipline' from '/home/rfit/Telescope_Array/phd_work/src/train_VAE/pipline.py'>

In [5]:
import pipline

1.10.1+cu111
Using device: cuda


/home/rfit/.conda/envs/robert_venv/lib/python3.6/site-packages/setuptools/distutils_patch.py:26: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


In [15]:
config = 'config.yaml'
model = pipline.Pipline(config)
exp = 'Fe_Pr18.01.2025_15:29latent_dim=16;_hidden_dim=512;_use_mask=True;_'
chpt = f'/home/rfit/Telescope_Array/phd_work/Models_VAE/{exp}/best'
model.load_chpt(chpt)

keys ['norm_param', 'test', 'train', 'val']
keys ['norm_param', 'test', 'train', 'val']
Encoder has params: 1097760 Decoder has params: 2134535


In [16]:
latent_list, params = model.predict_latent()

  0%|          | 1/697 [00:00<01:31,  7.57it/s]

keys ['norm_param', 'test', 'train', 'val']


100%|██████████| 697/697 [01:34<00:00,  7.41it/s]


torch.Size([1045262, 10])


In [17]:
import tensorboard as tb
import tensorflow as tf
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir=os.path.join('runs2', 'test1'))
writer.add_embedding(latent_list,
                        metadata=params[:,1],
                        )
writer.close()

In [22]:
import torch
fe_lat = latent_list[torch.where(params[:,1]>1000)]
pr_lat = latent_list[torch.where(params[:,1]<1000)]

In [24]:
print('fe', fe_lat.mean(), fe_lat.std())
print('pr', pr_lat.mean(), pr_lat.std())

fe tensor(0.9387) tensor(7.3101)
pr tensor(0.8348) tensor(6.6387)


In [30]:
for i in range(16):
    print(f'fe ch{i} mean', round(fe_lat[:,i].mean().item(),3),'std', round(fe_lat[:,i].std().item(),3))
    print(f'pr ch{i} mean', round(pr_lat[:,i].mean().item(),3),'std', round(pr_lat[:,i].std().item(),3), '\n')

fe ch0 mean 6.395 std 5.055
pr ch0 mean 5.75 std 4.47 

fe ch1 mean 6.023 std 3.139
pr ch1 mean 5.601 std 2.886 

fe ch2 mean 2.878 std 3.695
pr ch2 mean 2.517 std 3.499 

fe ch3 mean 8.874 std 5.427
pr ch3 mean 8.187 std 4.78 

fe ch4 mean -1.059 std 3.999
pr ch4 mean -1.163 std 3.79 

fe ch5 mean -2.031 std 3.939
pr ch5 mean -1.837 std 3.616 

fe ch6 mean 3.673 std 4.489
pr ch6 mean 3.147 std 4.004 

fe ch7 mean -1.272 std 3.315
pr ch7 mean -1.255 std 3.021 

fe ch8 mean -0.183 std 2.371
pr ch8 mean 0.055 std 2.161 

fe ch9 mean -0.975 std 3.142
pr ch9 mean -0.78 std 2.925 

fe ch10 mean 8.705 std 5.959
pr ch10 mean 8.051 std 5.123 

fe ch11 mean -10.094 std 7.754
pr ch11 mean -9.327 std 6.81 

fe ch12 mean -0.913 std 1.603
pr ch12 mean -0.906 std 1.516 

fe ch13 mean -10.639 std 7.942
pr ch13 mean -9.745 std 6.966 

fe ch14 mean -0.767 std 3.526
pr ch14 mean -0.736 std 3.254 

fe ch15 mean 6.405 std 4.462
pr ch15 mean 5.796 std 4.088 

